In [12]:
import heapq
import csv

file = './dataset/edge.csv' 
def dijkstra(graph, start, end):
    distances = {vertex: float('infinity') for vertex in graph}
    distances[start] = 0
    
    priority_queue = [(0, start)]
    
    while priority_queue:
        current_distance, current_vertex = heapq.heappop(priority_queue)
        
        if current_distance > distances[current_vertex]:
            continue

        if current_vertex == end:
            break
        
        for neighbor, weight in graph[current_vertex].items():
            distance = current_distance + weight
            
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                heapq.heappush(priority_queue, (distance, neighbor))
    
    return distances[end]

def convertCsvToNodes():
    graph = {}
    with open(file, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        
        for row in reader:
            place1 = row["place1"]
            mi_to_place = float(row["mi_to_place"])*1.609
            place2 = row["place2"]
            
            if place1 not in graph:
                graph[place1] = {}
            if place2 not in graph:
                graph[place2] = {}
            
            graph[place1][place2] = float(mi_to_place)
            graph[place2][place1] = float(mi_to_place)
    return graph

def findRowNames(start,end):
    ct = 0
    startNodeData = []
    endNodeData = []
    with open('./dataset/identifiers.csv', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if row['place'] == start:
                startNodeData = row
            if row['place'] == end:
                endNodeData = row
            if(len(startNodeData) > 0 and len(endNodeData) > 0):
                break


    return startNodeData, endNodeData

graph = convertCsvToNodes()

start = '00124'
end = '35800'
shortest_distance = dijkstra(graph, start, end)

placeStart , placeEnd = findRowNames(start, end)

if shortest_distance == float('infinity'):
    print(f"Não há caminho de {start} para {end}")
else:
    print(f"A distância mais curta do estado {placeStart['statename']} da cidade {placeStart['placename']} para estado {placeEnd['statename']} da cidade {placeEnd['placename']} é {round(shortest_distance, 2) } km")

A distância mais curta do estado Alabama da cidade Abbeville city para estado Alabama da cidade Homewood city é 224.91 km
